In [1]:
from sympy import *
from IPython.display import display, Markdown, Math
from math import comb, prod
import itertools
import random
init_session(use_latex=True)

IPython console for SymPy 1.9 (Python 3.9.7-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.9/



In [4]:
def Vertices(*args):
    return 2 + sum([args[i] * (i+1) for i in range(len(args))])

def Faces(*args):
    return sum(args)

def Edges(*args):
    return Vertices(*args) + Faces(*args) - 1

def R(*args): # args =[m_2, m_3, ...]
    # (Edges-1)!/((Edges-(m_2+m_3+m_4))! m_2! m_3! ...)
    vertices = Vertices(*args) # (2+m_2+2m_3+3m_4+...+(n-1)m_n)
    faces = Faces(*args) # (m_2+m_3+m_4+...+m_n)
    edges = vertices + faces - 1 # (1+2m_2+3m_3+4m_4+...+n m_n)
    r = factorial(edges-1)//factorial(edges-faces) # binomial(edges, faces) * (vertices-1)!/edges
    for m in args:
        r //= factorial(m)
    return r


def Binom(*args): # args =[m2, m3, ...]
    vertices = Vertices(*args)
    faces = Faces(*args)
    edges = vertices + faces - 1
    n = edges - 1
    r = 1
    for m in args: 
        r *= comb(n, m)
        n -= m
    r //= vertices - 1
    return r


def powers(n, k, c, terms = []):
    if not terms:
        terms = [0 for i in range(c+1)]
    if (k == n):
        terms[1] = -(-1)**k
        return [tuple(terms)]
    if c == 1:
        return []
    valids = powers(n, k, c-1, terms[::])
    while (c <= n) and (k > 0):
        terms[c] += 1
        n -= c
        k -= 1
        valids += powers(n, k, c-1, terms[::])
    return valids + powers(n, k, c-1, terms[::])
        
def diagonal(n, c, formulas):
    q = 0
    if not (2*n, n, len(c)-1) in formulas:
        formulas[(2*n, n, len(c)-1)] = set(powers(2*n, n, len(c)-1))
    p = formulas[(2*n, n, len(c)-1)]
    for el in p:
        term = el[1] * Binom(*el[2:]) * c[1]**(n-sum(el[2:]))
        term *= prod([c[k]**el[k] for k in range(2,len(el))])
        q += term
    return q

def sol(n, c, formulas):
    return sum([c[0]**(i+1)/c[1]**(2*i+1)*diagonal(i,c, formulas) for i in range(n+1)])

def algForm(n, c, formulas):
    q = 0
    if not (2*n, n, len(c)-1) in formulas:
        formulas[(2*n, n, len(c)-1)] = set(powers(2*n, n, len(c)-1))
    p = formulas[(2*n, n, len(c)-1)]
    for el in p:
        term = el[1] * Binom(*el[2:]) * Symbol('c_1')**(n-sum(el[2:]))
        tmp = 1
        for k in range(2,len(el)):
            c_ = 'c_'+str(k)
            Symbol(c_)
            tmp *= Symbol(c_)**el[k]
        q += term * tmp
    return q

def solForm(n,c, formulas):
    return sum([Symbol('c_0')**(i+1)/Symbol('c_1')**(2*i+1)*algForm(i,c, formulas) for i in range(n+1)]).expand()
        
def run():
    while True:
        steps = 0
        inp = input('Insert an expression with explicit math symbols (example: "x^5-10*x^3+.02*x^4+x**7"):\n')
        try:
            pol = Poly(sympify(inp), Symbol('x'))
            d = dict(pol.all_terms())
        except:
            print('Invalid expression. Turning off.')
            break
        values = [d[key] for key in sorted( d.keys())]
        if values[1] == 0:
            print('Invalid expression. Turn off.')
            break
        try:
            order = int(input('Insert the depth for the formula:\n'))
            assert(order >= 0)
        except:
            print('Invalid input (must be a natural number).')
            break
        try:
            guess = float(input('Please, insert your guess (or 0 otherwise):\n'))
        except:
            print('Invalid input (must be a decimal number).')
            break
        formulas = {}
        formula = solForm(order, values, formulas)
        display(
            Markdown(
                    "Using the formula "+\
                    '{}'.format(latex(formula, mode='inline')))
            )
        tmp = 0
        e = 0
        prv = .0 if guess else .1
        while abs(guess - prv) > .000000000000001:
            prv = guess
            steps += 1
            tmp = pol.as_expr()
            print('\n\n')
            display(
                Markdown(
                    'Bootstrapping from previous guess {}'.format(latex(guess, mode='inline'))
                )
            )
            print('Solving for', str(tmp).replace('x', '(x + '+str(guess)+')'), '= ', end = '')
            tmp = Poly(tmp, Symbol('x'))
            npol = Poly(pol.as_expr().replace(Symbol('x'), Symbol('x')+guess), Symbol('x'))
            d = dict(npol.all_terms())
            values = [d[key] for key in sorted( d.keys())]
            guess = float(guess+sol(order, values, formulas))
            e = tmp.eval(guess)
            display(
                Markdown(
                    '{}'.format(latex(npol.as_expr(), mode='inline'))
                ),
                Markdown(
                    'My new guess is p({})'.format(latex(guess, mode='inline'))+\
                    '= {}'.format(latex(e, mode='inline'))
                )
            )
            
        display(
            Markdown(
                    "Found a good approximation for p(x) = "+\
                    '{}'.format(latex(sympify(pol.as_expr()), mode='inline')))
            )
        display(N(guess))
        display(
            Markdown(
                'p({}) = '.format(latex(guess, mode='inline'))+\
                '{}'.format(latex(Float(e,30), mode='inline'))))
        print('Recursion:', steps, '\n\n\n\n\n')
            
# 3-4*x + x^2-2*x^3+x^4+3*x^5
#10.1+3.7*x+3*x**2-4*x^3-20.5*x**4+x**5+6.1*x**6-2.2*x**7-x**8+2.5*x^9-x**10
# x**5 + 0.6*x**4 - 7.16*x**3 + 1.8*x**2 + 4.72*x - 0.96

In [5]:
run()

Insert an expression with explicit math symbols (example: "x^5-10*x^3+.02*x^4+x**7"):
3-4*x + x^2-2*x^3+x^4+3*x^5
Insert the depth for the formula:
10
Please, insert your guess (or 0 otherwise):
9999


Using the formula $\frac{78 c_{0}^{11} c_{3} c_{5}^{2}}{c_{1}^{14}} + \frac{78 c_{0}^{11} c_{4}^{2} c_{5}}{c_{1}^{14}} - \frac{546 c_{0}^{11} c_{2}^{2} c_{5}^{2}}{c_{1}^{15}} - \frac{2184 c_{0}^{11} c_{2} c_{3} c_{4} c_{5}}{c_{1}^{15}} - \frac{364 c_{0}^{11} c_{2} c_{4}^{3}}{c_{1}^{15}} - \frac{364 c_{0}^{11} c_{3}^{3} c_{5}}{c_{1}^{15}} - \frac{546 c_{0}^{11} c_{3}^{2} c_{4}^{2}}{c_{1}^{15}} + \frac{5460 c_{0}^{11} c_{2}^{3} c_{4} c_{5}}{c_{1}^{16}} + \frac{8190 c_{0}^{11} c_{2}^{2} c_{3}^{2} c_{5}}{c_{1}^{16}} + \frac{8190 c_{0}^{11} c_{2}^{2} c_{3} c_{4}^{2}}{c_{1}^{16}} + \frac{5460 c_{0}^{11} c_{2} c_{3}^{3} c_{4}}{c_{1}^{16}} + \frac{273 c_{0}^{11} c_{3}^{5}}{c_{1}^{16}} - \frac{21840 c_{0}^{11} c_{2}^{4} c_{3} c_{5}}{c_{1}^{17}} - \frac{10920 c_{0}^{11} c_{2}^{4} c_{4}^{2}}{c_{1}^{17}} - \frac{43680 c_{0}^{11} c_{2}^{3} c_{3}^{2} c_{4}}{c_{1}^{17}} - \frac{10920 c_{0}^{11} c_{2}^{2} c_{3}^{4}}{c_{1}^{17}} + \frac{12376 c_{0}^{11} c_{2}^{6} c_{5}}{c_{1}^{18}} + \frac{74256 c_{0}^{11} c_{2}^{5} c_{3} c_{4}}{c_{1}^{18}} + \frac{61880 c_{0}^{11} c_{2}^{4} c_{3}^{3}}{c_{1}^{18}} - \frac{31824 c_{0}^{11} c_{2}^{7} c_{4}}{c_{1}^{19}} - \frac{111384 c_{0}^{11} c_{2}^{6} c_{3}^{2}}{c_{1}^{19}} + \frac{75582 c_{0}^{11} c_{2}^{8} c_{3}}{c_{1}^{20}} - \frac{16796 c_{0}^{11} c_{2}^{10}}{c_{1}^{21}} - \frac{66 c_{0}^{10} c_{2} c_{5}^{2}}{c_{1}^{13}} - \frac{132 c_{0}^{10} c_{3} c_{4} c_{5}}{c_{1}^{13}} - \frac{22 c_{0}^{10} c_{4}^{3}}{c_{1}^{13}} + \frac{858 c_{0}^{10} c_{2}^{2} c_{4} c_{5}}{c_{1}^{14}} + \frac{858 c_{0}^{10} c_{2} c_{3}^{2} c_{5}}{c_{1}^{14}} + \frac{858 c_{0}^{10} c_{2} c_{3} c_{4}^{2}}{c_{1}^{14}} + \frac{286 c_{0}^{10} c_{3}^{3} c_{4}}{c_{1}^{14}} - \frac{4004 c_{0}^{10} c_{2}^{3} c_{3} c_{5}}{c_{1}^{15}} - \frac{2002 c_{0}^{10} c_{2}^{3} c_{4}^{2}}{c_{1}^{15}} - \frac{6006 c_{0}^{10} c_{2}^{2} c_{3}^{2} c_{4}}{c_{1}^{15}} - \frac{1001 c_{0}^{10} c_{2} c_{3}^{4}}{c_{1}^{15}} + \frac{3003 c_{0}^{10} c_{2}^{5} c_{5}}{c_{1}^{16}} + \frac{15015 c_{0}^{10} c_{2}^{4} c_{3} c_{4}}{c_{1}^{16}} + \frac{10010 c_{0}^{10} c_{2}^{3} c_{3}^{3}}{c_{1}^{16}} - \frac{8008 c_{0}^{10} c_{2}^{6} c_{4}}{c_{1}^{17}} - \frac{24024 c_{0}^{10} c_{2}^{5} c_{3}^{2}}{c_{1}^{17}} + \frac{19448 c_{0}^{10} c_{2}^{7} c_{3}}{c_{1}^{18}} - \frac{4862 c_{0}^{10} c_{2}^{9}}{c_{1}^{19}} - \frac{5 c_{0}^{9} c_{5}^{2}}{c_{1}^{11}} + \frac{110 c_{0}^{9} c_{2} c_{4} c_{5}}{c_{1}^{12}} + \frac{55 c_{0}^{9} c_{3}^{2} c_{5}}{c_{1}^{12}} + \frac{55 c_{0}^{9} c_{3} c_{4}^{2}}{c_{1}^{12}} - \frac{660 c_{0}^{9} c_{2}^{2} c_{3} c_{5}}{c_{1}^{13}} - \frac{330 c_{0}^{9} c_{2}^{2} c_{4}^{2}}{c_{1}^{13}} - \frac{660 c_{0}^{9} c_{2} c_{3}^{2} c_{4}}{c_{1}^{13}} - \frac{55 c_{0}^{9} c_{3}^{4}}{c_{1}^{13}} + \frac{715 c_{0}^{9} c_{2}^{4} c_{5}}{c_{1}^{14}} + \frac{2860 c_{0}^{9} c_{2}^{3} c_{3} c_{4}}{c_{1}^{14}} + \frac{1430 c_{0}^{9} c_{2}^{2} c_{3}^{3}}{c_{1}^{14}} - \frac{2002 c_{0}^{9} c_{2}^{5} c_{4}}{c_{1}^{15}} - \frac{5005 c_{0}^{9} c_{2}^{4} c_{3}^{2}}{c_{1}^{15}} + \frac{5005 c_{0}^{9} c_{2}^{6} c_{3}}{c_{1}^{16}} - \frac{1430 c_{0}^{9} c_{2}^{8}}{c_{1}^{17}} + \frac{9 c_{0}^{8} c_{4} c_{5}}{c_{1}^{10}} - \frac{90 c_{0}^{8} c_{2} c_{3} c_{5}}{c_{1}^{11}} - \frac{45 c_{0}^{8} c_{2} c_{4}^{2}}{c_{1}^{11}} - \frac{45 c_{0}^{8} c_{3}^{2} c_{4}}{c_{1}^{11}} + \frac{165 c_{0}^{8} c_{2}^{3} c_{5}}{c_{1}^{12}} + \frac{495 c_{0}^{8} c_{2}^{2} c_{3} c_{4}}{c_{1}^{12}} + \frac{165 c_{0}^{8} c_{2} c_{3}^{3}}{c_{1}^{12}} - \frac{495 c_{0}^{8} c_{2}^{4} c_{4}}{c_{1}^{13}} - \frac{990 c_{0}^{8} c_{2}^{3} c_{3}^{2}}{c_{1}^{13}} + \frac{1287 c_{0}^{8} c_{2}^{5} c_{3}}{c_{1}^{14}} - \frac{429 c_{0}^{8} c_{2}^{7}}{c_{1}^{15}} - \frac{8 c_{0}^{7} c_{3} c_{5}}{c_{1}^{9}} - \frac{4 c_{0}^{7} c_{4}^{2}}{c_{1}^{9}} + \frac{36 c_{0}^{7} c_{2}^{2} c_{5}}{c_{1}^{10}} + \frac{72 c_{0}^{7} c_{2} c_{3} c_{4}}{c_{1}^{10}} + \frac{12 c_{0}^{7} c_{3}^{3}}{c_{1}^{10}} - \frac{120 c_{0}^{7} c_{2}^{3} c_{4}}{c_{1}^{11}} - \frac{180 c_{0}^{7} c_{2}^{2} c_{3}^{2}}{c_{1}^{11}} + \frac{330 c_{0}^{7} c_{2}^{4} c_{3}}{c_{1}^{12}} - \frac{132 c_{0}^{7} c_{2}^{6}}{c_{1}^{13}} + \frac{7 c_{0}^{6} c_{2} c_{5}}{c_{1}^{8}} + \frac{7 c_{0}^{6} c_{3} c_{4}}{c_{1}^{8}} - \frac{28 c_{0}^{6} c_{2}^{2} c_{4}}{c_{1}^{9}} - \frac{28 c_{0}^{6} c_{2} c_{3}^{2}}{c_{1}^{9}} + \frac{84 c_{0}^{6} c_{2}^{3} c_{3}}{c_{1}^{10}} - \frac{42 c_{0}^{6} c_{2}^{5}}{c_{1}^{11}} + \frac{c_{0}^{5} c_{5}}{c_{1}^{6}} - \frac{6 c_{0}^{5} c_{2} c_{4}}{c_{1}^{7}} - \frac{3 c_{0}^{5} c_{3}^{2}}{c_{1}^{7}} + \frac{21 c_{0}^{5} c_{2}^{2} c_{3}}{c_{1}^{8}} - \frac{14 c_{0}^{5} c_{2}^{4}}{c_{1}^{9}} - \frac{c_{0}^{4} c_{4}}{c_{1}^{5}} + \frac{5 c_{0}^{4} c_{2} c_{3}}{c_{1}^{6}} - \frac{5 c_{0}^{4} c_{2}^{3}}{c_{1}^{7}} + \frac{c_{0}^{3} c_{3}}{c_{1}^{4}} - \frac{2 c_{0}^{3} c_{2}^{2}}{c_{1}^{5}} - \frac{c_{0}^{2} c_{2}}{c_{1}^{3}} - c_{0} / c_{1}$

Bootstrapping from previous guess $9999.0$

Solving for 3*(x + 9999.0)**5 + (x + 9999.0)**4 - 2*(x + 9999.0)**3 + (x + 9999.0)**2 - 4*(x + 9999.0) + 3 = 

$3.0 x^{5} + 149986.0 x^{4} + 2999440024.0 x^{3} + 29991600719983.0 x^{2} + 1.4994400719966 \cdot 10^{17} x + 2.998600239983 \cdot 10^{20}$

...

$1.0 x^{5} - 14.4 x^{4} + 75.6400000000001 x^{3} - 171.36 x^{2} + 140.8 x - 1.13686837721616 \cdot 10^{-13}$

My new guess is p($-3.0$)= $2.48689957516035 \cdot 10^{-14}$

Found a good approximation for p(x) = $1.0 x^{5} + 0.6 x^{4} - 7.16 x^{3} + 1.8 x^{2} + 4.72 x - 0.96$

p($-3.0$) = $2.48689957516035065054893493652 \cdot 10^{-14}$

Recursion: 12 





Insert an expression with explicit math symbols (example: "x^5-10*x^3+.02*x^4+x**7"):
2d
Invalid expression. Turning off.
